# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint


# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output.data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,4.01,100,100,2.06,GL,1689535893
1,1,kudahuvadhoo,2.6708,72.8944,28.20,76,61,4.58,MV,1689535894
2,2,grytviken,-54.2811,-36.5092,-5.85,70,18,2.11,GS,1689535894
3,3,slatyne,50.2242,36.1608,17.22,75,0,3.48,UA,1689535894
4,4,wajir,1.7471,40.0573,27.58,42,34,8.94,KE,1689535341


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
   "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .8,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df["Max Temp"] > 20)&(city_data_df["Max Temp"] < 35) & (city_data_df["Humidity"] < 55)]


# Drop any rows with null values
criteria_df = criteria_df.dropna()

# Display sample data
criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,wajir,1.7471,40.0573,27.58,42,34,8.94,KE,1689535341
12,12,mammoth lakes,37.6485,-118.9721,31.25,13,0,1.79,US,1689535898
17,17,tumayr,25.7039,45.8680,33.63,7,0,4.51,SA,1689535900
23,23,whitehorse,60.7161,-135.0538,22.83,40,20,4.63,CA,1689535664
27,27,aldama,22.9167,-98.0667,34.72,35,0,6.30,MX,1689535904
...,...,...,...,...,...,...,...,...,...,...
544,544,rice lake,45.5061,-91.7382,24.82,53,75,7.20,US,1689535891
550,550,atasu,48.6814,71.6433,22.35,36,7,4.00,KZ,1689536121
556,556,colonia,50.9333,6.9500,22.70,44,0,4.12,DE,1689535899
561,561,orvieto,42.7195,12.1116,28.64,51,0,1.66,IT,1689536125


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= hotel_df = criteria_df[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df=hotel_df.assign(Hotel_Name="")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel_Name
4,wajir,KE,1.7471,40.0573,42,
12,mammoth lakes,US,37.6485,-118.9721,13,
17,tumayr,SA,25.7039,45.8680,7,
23,whitehorse,CA,60.7161,-135.0538,40,
27,aldama,MX,22.9167,-98.0667,35,
...,...,...,...,...,...,...
544,rice lake,US,45.5061,-91.7382,53,
550,atasu,KZ,48.6814,71.6433,36,
556,colonia,DE,50.9333,6.9500,44,
561,orvieto,IT,42.7195,12.1116,51,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
# Set parameters to search for a hotel
radius = 10000
limit = 1

params = {
    "categories":"accommodation.hotel",
    "limit":limit,
    "apiKey":geoapify_key 
}



# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude =  row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wajir - nearest hotel: Kilimanjaro
mammoth lakes - nearest hotel: Travelodge by Wyndham Mammoth Lakes
tumayr - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
aldama - nearest hotel: Presas del rey
'ain el melh - nearest hotel: No hotel found
kashgar - nearest hotel: 深航国际酒店
laojunmiao - nearest hotel: No hotel found
dayrut - nearest hotel: No hotel found
fort mcmurray - nearest hotel: Nomad Hotel and Suites
hato mayor del rey - nearest hotel: dormitorio hato mayor
yellowknife - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
torghay - nearest hotel: No hotel found
la sarre - nearest hotel: No hotel found
urucara - nearest hotel: No hotel found
almeirim - nearest hotel: Kapejar Hotel
yeed - nearest hotel: No hotel found
twin falls - nearest hotel: No hotel found
lodja - nearest hotel: No hotel found
el battan - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
ghormach - neares

,City,Country,Lat,Lng,Humidity,Hotel_Name,Hotel Name
4,wajir,KE,1.7471,40.0573,42,,Kilimanjaro
12,mammoth lakes,US,37.6485,-118.9721,13,,Travelodge by Wyndham Mammoth Lakes
17,tumayr,SA,25.7039,45.8680,7,,No hotel found
23,whitehorse,CA,60.7161,-135.0538,40,,Town & Mountain Hotel
27,aldama,MX,22.9167,-98.0667,35,,Presas del rey
...,...,...,...,...,...,...,...
544,rice lake,US,45.5061,-91.7382,53,,Paul Bunyan Resort
550,atasu,KZ,48.6814,71.6433,36,,Диана
556,colonia,DE,50.9333,6.9500,44,,Wasserturm Hotel Cologne
561,orvieto,IT,42.7195,12.1116,51,,Hotel Royal


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
   "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = .8,
    color = "City",
    hover_data=['City', 'Hotel Name', 'Country'],
    hover_cols=['City', 'Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)